# Introduction

This notebook calculates various values such as the total number of trips for each year-month.

The data has been prepared by `/notebooks/1_etl.ipynb` and is saved in parquet format in `/data/processed/cleaned`.

In [89]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
import pandas as pd
from pathlib import Path
from dotenv import find_dotenv
from pyspark.sql import SparkSession
from datetime import datetime
import pyspark.sql.functions as F
from pyspark.sql.types import (
    IntegerType,
    DateType,
    FloatType,
    StringType
)

from src.utils.utils import write_sql_to_file

In [91]:
pd.set_option('display.max_columns', 100)

In [92]:
project_dir = Path(find_dotenv()).cwd().parent
data_dir = project_dir / 'data'
raw_data_dir = data_dir / 'raw'
processed_data_dir = data_dir / 'processed'
reports_dir = project_dir / 'reports'

In [93]:
spark = (
    SparkSession
    .builder
    .master('local[12]')
    .appName('new_york_taxis')
    .getOrCreate()
)

In [94]:
spark

In [95]:
spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

'3.0.0'

In [96]:
spark.sparkContext._conf.getAll()

[('spark.driver.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.executor.memory', '16g'),
 ('spark.app.name', 'new_york_taxis'),
 ('spark.executor.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.master', 'local[12]'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '7501c46205ca'),
 ('spark.driver.port', '36583'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1619227295815'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.debug.maxToStringFields', '1000')]

In [97]:
conf = spark.sparkContext._conf.setAll([
    ('spark.driver.memory', '16g'),
    ('spark.executor.memory', '16g'),
    ('spark.app.name', 'new_york_taxis'),
])
spark.sparkContext.stop()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [98]:
spark.sparkContext._conf.getAll()

[('spark.driver.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.executor.memory', '16g'),
 ('spark.app.name', 'new_york_taxis'),
 ('spark.executor.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.master', 'local[12]'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '7501c46205ca'),
 ('spark.driver.port', '36583'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1619227295815'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.debug.maxToStringFields', '1000')]

# Load data

In [99]:
path = processed_data_dir.joinpath('df_cleaned').as_posix()
df = spark.read.parquet(path)

In [100]:
df.count()

115510730

In [101]:
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- Store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- Passenger_count: integer (nullable = true)
 |-- Fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- Total_amount: float (nullable = true)
 |-- Payment_type: string (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- colour: string (nullable = true)
 |-- pickup_year: integer (nullable = true)
 |-- pickup_month: integer (nullable = true)
 |-- pickup_dayofyear: integer (nullable = true)
 |-- pickup_dayofmonth: integer (nullable = true)
 |-- picku

In [102]:
df.limit(10).toPandas()

,VendorID,pickup_datetime,dropoff_datetime,Store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,Passenger_count,Fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,Total_amount,Payment_type,congestion_surcharge,colour,pickup_year,pickup_month,pickup_dayofyear,pickup_dayofmonth,pickup_dayofweek,pickup_weekofyear,pickup_hourofday,dropoff_year,trip_duration,trip_distance_km,speed
0,2,2019-05-20 17:58:02,2019-05-20 18:31:17,N,1,40,80,1.0,23.0,1.0,0.5,0.0,0.00,0.3,24.799999,2,0.0,green,2019,5,140,20,2,21,17,2019,1995,8.915744,16.088560
1,2,2019-07-12 07:06:55,2019-07-12 07:10:15,None,1.40,1,N,NaN,1.0,6.0,0.0,0.5,1.50,0.0,0.300000,None,2.5,yellow,2019,7,193,12,6,28,7,2019,200,1.353099,24.355786
2,2,2019-05-06 08:16:49,2019-05-06 08:29:24,None,1.91,1,N,NaN,1.0,10.0,0.0,0.5,1.00,0.0,0.300000,None,2.5,yellow,2019,5,126,6,2,19,8,2019,755,5.107950,24.355786
3,2,2019-06-21 00:48:59,2019-06-21 01:04:28,None,3.60,1,N,NaN,1.0,14.5,0.5,0.5,3.66,0.0,0.300000,None,2.5,yellow,2019,6,172,21,6,25,0,2019,929,6.285146,24.355786
4,1,2019-02-15 23:26:23,2019-02-15 23:32:49,None,.80,1,N,NaN,1.0,6.0,3.0,0.5,1.95,0.0,0.300000,None,2.5,yellow,2019,2,46,15,6,7,23,2019,386,2.611482,24.355786
5,2,2020-01-23 22:05:30,2020-01-23 22:18:12,None,2.55,1,N,NaN,1.0,11.0,0.5,0.5,2.96,0.0,0.300000,None,2.5,yellow,2020,1,23,23,5,4,22,2020,762,5.155308,24.355786
6,2,2020-02-29 01:46:32,2020-02-29 02:03:15,None,5.04,1,N,NaN,2.0,18.0,0.5,0.5,0.00,0.0,0.300000,None,2.5,yellow,2020,2,60,29,7,9,1,2020,1003,6.785793,24.355786
7,2,2019-07-22 09:01:21,2019-07-22 09:09:30,None,.87,1,N,NaN,1.0,7.0,0.0,0.5,2.06,0.0,0.300000,None,2.5,yellow,2019,7,203,22,2,30,9,2019,489,3.308328,24.355786
8,1,2019-10-27 11:36:14,2019-10-27 11:44:31,None,1.50,1,N,NaN,1.0,7.5,2.5,0.5,1.62,0.0,0.300000,None,2.5,yellow,2019,10,300,27,1,43,11,2019,497,3.362452,24.355786
9,1,2019-12-12 20:22:45,2019-12-12 20:29:27,None,1.80,1,N,NaN,1.0,7.5,3.0,0.5,1.13,0.0,0.300000,None,2.5,yellow,2019,12,346,12,5,50,20,2019,402,2.719729,24.355786


# SQL file

Create a file in the `reports_dir` to store all the sql queries used in the investigations.

In [103]:
sql_path = reports_dir / 'task_4.sql'
with open(sql_path, 'w+') as f:
    f.write('-- MDSI BDE AT1 Task 4 SQL Queries\n')

# Tasks

The following tasks require using SQL queries.

In [104]:
# This registers the table "df" so that it can be referenced in SQL queries.
df.createOrReplaceTempView('df')

## a. For each month year and month

### i. What was the total number of trips?

In [105]:
comment = 'a.i. What was the total number of trips?'

In [106]:
query = """
SELECT 
  pickup_year
, pickup_month
, COUNT(*) AS n_trips
FROM df
GROUP BY 1, 2
ORDER BY 1, 2
"""

spark.sql(query).show(50)

+-----------+------------+-------+
|pickup_year|pickup_month|n_trips|
+-----------+------------+-------+
|       2019|           1|8210354|
|       2019|           2|7515946|
|       2019|           3|8348950|
|       2019|           4|7869061|
|       2019|           5|7985822|
|       2019|           6|7327653|
|       2019|           7|6701176|
|       2019|           8|6443544|
|       2019|           9|6934948|
|       2019|          10|7606108|
|       2019|          11|7246815|
|       2019|          12|7265200|
|       2020|           1|6776129|
|       2020|           2|6626305|
|       2020|           3|3194072|
|       2020|           4| 268174|
|       2020|           5| 398037|
|       2020|           6| 603136|
|       2020|           7| 859777|
|       2020|           8|1073624|
|       2020|           9|1407156|
|       2020|          10|1751017|
|       2020|          11|1575114|
|       2020|          12|1522612|
+-----------+------------+-------+



In [107]:
write_sql_to_file(comment, query, sql_path)

### ii. Which weekday had the most trips?

In [108]:
comment = 'a.ii. Which weekday had the most trips?'

In [109]:
var = 'pickup_dayofweek'

query = f"""
SELECT 
  pickup_year
, pickup_month
, FIRST({var}) AS {var}
, MAX(n_trips)

FROM (
    SELECT 
      pickup_year
    , pickup_month
    , {var}
    , COUNT(*) AS n_trips
    FROM df
    GROUP BY 1, 2, 3
    ORDER BY 1, 2, 4 DESC
    ) temp

GROUP BY 1, 2
ORDER BY 1, 2
"""

spark.sql(query).show(2 * 12)

+-----------+------------+----------------+------------+
|pickup_year|pickup_month|pickup_dayofweek|max(n_trips)|
+-----------+------------+----------------+------------+
|       2019|           1|               5|     1446731|
|       2019|           2|               6|     1187928|
|       2019|           3|               6|     1471918|
|       2019|           4|               3|     1331567|
|       2019|           5|               5|     1398005|
|       2019|           6|               7|     1202971|
|       2019|           7|               4|     1186217|
|       2019|           8|               5|     1162471|
|       2019|           9|               5|     1044398|
|       2019|          10|               5|     1326859|
|       2019|          11|               6|     1283342|
|       2019|          12|               3|     1148349|
|       2020|           1|               6|     1193138|
|       2020|           2|               7|     1127510|
|       2020|           3|     

In [110]:
write_sql_to_file(comment, query, sql_path)

### iii. Which hour of the day had the most trips?

In [111]:
comment = 'a.iii. Which hour of the day had the most trips?'

In [112]:
var = 'pickup_hourofday'

query = f"""
SELECT 
  pickup_year
, pickup_month
, FIRST({var}) AS {var}
, MAX(n_trips)

FROM (
    SELECT 
      pickup_year
    , pickup_month
    , {var}
    , COUNT(*) AS n_trips
    FROM df
    GROUP BY 1, 2, 3
    ORDER BY 1, 2, 4 DESC
    ) temp

GROUP BY 1, 2
ORDER BY 1, 2
"""

spark.sql(query).show(2 * 12)

+-----------+------------+----------------+------------+
|pickup_year|pickup_month|pickup_hourofday|max(n_trips)|
+-----------+------------+----------------+------------+
|       2019|           1|              18|      553055|
|       2019|           2|              18|      508192|
|       2019|           3|              18|      554505|
|       2019|           4|              18|      531193|
|       2019|           5|              18|      528350|
|       2019|           6|              18|      462616|
|       2019|           7|              18|      439385|
|       2019|           8|              18|      421839|
|       2019|           9|              18|      458833|
|       2019|          10|              18|      508398|
|       2019|          11|              18|      476668|
|       2019|          12|              18|      461589|
|       2020|           1|              18|      462257|
|       2020|           2|              18|      462052|
|       2020|           3|     

In [113]:
write_sql_to_file(comment, query, sql_path)

### iv. What was the average number of passengers?

In [114]:
comment = 'a.iv. What was the average number of passengers?'

In [115]:
var = 'Passenger_count'

query = f"""
SELECT 
  pickup_year
, pickup_month
, AVG({var}) AS mean_{var}
FROM df
GROUP BY 1, 2
ORDER BY 1, 2
"""

spark.sql(query).show(2 * 12)

+-----------+------------+--------------------+
|pickup_year|pickup_month|mean_Passenger_count|
+-----------+------------+--------------------+
|       2019|           1|  1.3079210440688014|
|       2019|           2|  1.3040477003863506|
|       2019|           3|  1.3050554377379924|
|       2019|           4|   1.306148339449957|
|       2019|           5|  1.3034665193936192|
|       2019|           6|  1.3100697606488083|
|       2019|           7|   1.295917817273279|
|       2019|           8|  1.2942384681445314|
|       2019|           9|  1.3195596002781036|
|       2019|          10|   1.315669542410045|
|       2019|          11|  1.3260138702112763|
|       2019|          12|  1.3321549347648969|
|       2020|           1|   1.322345475683007|
|       2020|           2|  1.3345820089803744|
|       2020|           3|   1.309581027214591|
|       2020|           4|  1.1434426229508197|
|       2020|           5|  1.1617226471566344|
|       2020|           6|  1.1887505437

In [116]:
write_sql_to_file(comment, query, sql_path)

In [117]:
df.select('Passenger_count').describe().show()

+-------+------------------+
|summary|   Passenger_count|
+-------+------------------+
|  count|           6752718|
|   mean|1.3071554298580217|
| stddev|0.8770972573832002|
|    min|                 1|
|    max|                 5|
+-------+------------------+



In [118]:
#
df.groupby('Passenger_count').count().sort(F.col('Passenger_count')).show()

+---------------+---------+
|Passenger_count|    count|
+---------------+---------+
|           null|108758012|
|              1|  5770149|
|              2|   485748|
|              3|    94673|
|              4|   209552|
|              5|   192596|
+---------------+---------+



### v. What was the average amount paid per trip (total_amount)?

In [119]:
commend = 'a.v. What was the average amount paid per trip (total_amount)?'

In [120]:
df.select('Total_amount').describe().show()

+-------+------------------+
|summary|      Total_amount|
+-------+------------------+
|  count|         115510730|
|   mean|1.4713562514553544|
| stddev| 5.751008121661385|
|    min|               0.0|
|    max|            4012.3|
+-------+------------------+



In [121]:
var = 'Total_amount'

query = f"""
SELECT 
  pickup_year
, pickup_month
, AVG({var}) AS mean_{var}
FROM df
GROUP BY 1, 2
ORDER BY 1, 2
"""

spark.sql(query).show(2 * 12)

+-----------+------------+------------------+
|pickup_year|pickup_month| mean_Total_amount|
+-----------+------------+------------------+
|       2019|           1|1.5017026748905267|
|       2019|           2|1.5506898292641265|
|       2019|           3|1.4699229495408608|
|       2019|           4|1.2943942740185483|
|       2019|           5|1.2854630235164666|
|       2019|           6|1.3041023026697547|
|       2019|           7|1.4835741251733086|
|       2019|           8|1.5280360287694674|
|       2019|           9|1.4679342299132532|
|       2019|          10|1.4565563429228852|
|       2019|          11|1.4320420873808115|
|       2019|          12|1.4290651260134435|
|       2020|           1|1.5168927096041134|
|       2020|           2|1.3844046444225515|
|       2020|           3|1.4343413394770392|
|       2020|           4| 2.837826455427217|
|       2020|           5| 3.294334151852783|
|       2020|           6|2.6476867651919274|
|       2020|           7| 2.28727

In [122]:
write_sql_to_file(comment, query, sql_path)

### vi. What was the average amount paid per passenger (total_amount)?

In [123]:
comment = 'a.vi. What was the average amount paid per passenger (total_amount)?'

In [124]:
var1 = 'Passenger_count'
var2 = 'Total_amount'

query = f"""
SELECT 
  pickup_year
, pickup_month
, SUM({var1}) / SUM({var2}) AS average_amount_per_passenger
FROM df
GROUP BY 1, 2
ORDER BY 1, 2
"""

spark.sql(query).show(2 * 12)

+-----------+------------+----------------------------+
|pickup_year|pickup_month|average_amount_per_passenger|
+-----------+------------+----------------------------+
|       2019|           1|         0.06585500663726629|
|       2019|           2|         0.06353893514505982|
|       2019|           3|          0.0632818757750124|
|       2019|           4|         0.06509578287140212|
|       2019|           5|         0.06345636738095163|
|       2019|           6|         0.06382978400249055|
|       2019|           7|         0.05561578831016162|
|       2019|           8|        0.051582133084832955|
|       2019|           9|        0.049965189004116416|
|       2019|          10|         0.04554937564792602|
|       2019|          11|         0.04598716217568497|
|       2019|          12|        0.045985222953065875|
|       2020|           1|        0.042493473967264314|
|       2020|           2|          0.0461377424684784|
|       2020|           3|        0.049839027975

In [125]:
write_sql_to_file(comment, query, sql_path)

## b. For each taxi colour (yellow or green)

### i. What was the average, median, minimum and maximum trip duration in seconds?


In [126]:
comment = 'b.i. What was the average, median, minimum and maximum trip duration in seconds?'

In [127]:
var1 = 'colour'
var2 = 'trip_duration'

query = f"""
SELECT 
  {var1}
, AVG({var2}) AS average_trip_duration
, PERCENTILE({var2}, 0.5) AS median_trip_duration
, MIN({var2}) AS mininum_trip_duration
, MAX({var2}) AS maximum_trip_duration
FROM df
GROUP BY 1
"""

spark.sql(query).show(2 * 12)

+------+---------------------+--------------------+---------------------+---------------------+
|colour|average_trip_duration|median_trip_duration|mininum_trip_duration|maximum_trip_duration|
+------+---------------------+--------------------+---------------------+---------------------+
| green|    1312.577714441537|               727.0|                   61|               589227|
|yellow|   1059.8674022814157|               669.0|                   61|              2618881|
+------+---------------------+--------------------+---------------------+---------------------+



In [128]:
write_sql_to_file(comment, query, sql_path)

### ii. What was the average, median, minimum and maximum trip distance in km?

In [129]:
comment = 'b.ii. What was the average, median, minimum and maximum trip distance in km?'

In [130]:
var1 = 'colour'
var2 = 'trip_distance_km'

query = f"""
SELECT 
  {var1}
, AVG({var2}) AS average_trip_distance_km
, PERCENTILE({var2}, 0.5) AS median_trip_distance_km
, MIN({var2}) AS mininum_trip_distance_km
, MAX({var2}) AS maximum_trip_distance_km
FROM df
GROUP BY 1
"""

spark.sql(query).show(2 * 12)

+------+------------------------+-----------------------+------------------------+------------------------+
|colour|average_trip_distance_km|median_trip_distance_km|mininum_trip_distance_km|maximum_trip_distance_km|
+------+------------------------+-----------------------+------------------------+------------------------+
| green|       5.696300922784143|     3.3152403079128265|                     0.0|       830.2424361743164|
|yellow|       6.585289523121792|       4.53288246589814|     0.41269526928326355|       426.4751000000001|
+------+------------------------+-----------------------+------------------------+------------------------+



In [131]:
write_sql_to_file(comment, query, sql_path)

### iii. What was the average, median, minimum and maximum speed in km per hour?

In [132]:
comment = 'b.iii. What was the average, median, minimum and maximum speed in km per hour?'

In [133]:
var1 = 'colour'
var2 = 'speed'

query = f"""
SELECT 
  {var1}
, AVG({var2}) AS average_speed
, PERCENTILE({var2}, 0.5) AS median_speed
, MIN({var2}) AS mininum_speed
, MAX({var2}) AS maximum_speed
FROM df
GROUP BY 1
"""

spark.sql(query).show(2 * 12)

+------+-----------------+------------------+-------------------+-----------------+
|colour|    average_speed|      median_speed|      mininum_speed|    maximum_speed|
+------+-----------------+------------------+-------------------+-----------------+
| green|19.65947684205493| 17.54407296788189|                0.0|79.99402750674095|
|yellow|24.68783249710792|24.355786383930308|0.06706126653779819|79.99982927977405|
+------+-----------------+------------------+-------------------+-----------------+



In [134]:
write_sql_to_file(comment, query, sql_path)

## c. What was the percentage of trips where the driver received tips?

In [135]:
comment = 'c. What was the percentage of trips where the driver received tips?'

In [136]:
var1 = 'colour'
var2 = 'tip_amount'

query = f"""
SELECT 
  {var1}
, AVG(tip_ind) AS perc_received_tips
FROM (
    SELECT 
      {var1}
    , CASE WHEN {var2} > 0 THEN 1 ELSE 0 END AS tip_ind
    FROM df
    ) temp
GROUP BY 1
"""

spark.sql(query).show(2 * 12)

+------+-------------------+
|colour| perc_received_tips|
+------+-------------------+
| green|0.38053061727488685|
|yellow|  0.993257029072511|
+------+-------------------+



In [137]:
write_sql_to_file(comment, query, sql_path)

## d. For trips where the driver received tips, what was the percentage where the driver received tips of at least $10.

0.0313252%

In [138]:
comment = 'd. For trips where the driver received tips, what was the percentage where the driver received tips of at least $10.'

In [139]:
var = 'tip_amount'

query = f"""

SELECT 
  CASE WHEN {var} >= 10 THEN 1 ELSE 0 END AS gt_10_ind
, COUNT(*) AS n_trips
, FORMAT_STRING('%.6e', COUNT(*) / SUM(COUNT(*)) OVER ()) AS perc_trips

FROM df
GROUP BY 1
"""

spark.sql(query).show(2 * 12)

+---------+---------+------------+
|gt_10_ind|  n_trips|  perc_trips|
+---------+---------+------------+
|        1|    36184|3.132523e-04|
|        0|115474546|9.996867e-01|
+---------+---------+------------+



In [140]:
write_sql_to_file(comment, query, sql_path)

## e. Classify each trip into bins of durations:

1. Under 5 Mins
1. From 5 mins to 10 mins
1. From 10 mins to 20 mins
1. From 20 mins to 30 mins
1. At least 30 mins

Then for each bins, calculate:
* Average speed (km per hour)
* Average distance per dollar (km per $)

In [141]:
comment = 'e. Classify each trip into bins of durations'

In [142]:
var1 = 'trip_duration'
var2 = 'speed'
var3 = 'trip_distance_km'
var4 = 'Total_amount'

query = f"""
SELECT 
  CASE
    WHEN {var1} < 5 * 60 THEN 'lt_5'
    WHEN {var1} >= 5 * 60 AND {var1} < 10 * 60 THEN 'bt_5_10'
    WHEN {var1} >= 10 * 60 AND {var1} < 20 * 60 THEN 'bt_10_20'
    WHEN {var1} >= 20 * 60 AND {var1} < 30 * 60 THEN 'bt_20_30'
    WHEN {var1} >= 30 * 60 THEN 'gt_30'
  END as duration_group
, AVG({var2}) AS avg_speed
, SUM({var3}) / SUM({var4}) AS km_dollar
FROM df
GROUP BY 1
ORDER BY 3
"""

spark.sql(query).show(2 * 12)

+--------------+------------------+------------------+
|duration_group|         avg_speed|         km_dollar|
+--------------+------------------+------------------+
|          lt_5| 24.03169394044406| 2.071765560065408|
|       bt_5_10|24.085732398608357|3.3834257482780172|
|      bt_10_20|24.389322722193143| 4.384531926774909|
|      bt_20_30|24.893936704035198| 4.634846452236881|
|         gt_30| 24.87902416809538| 5.670214286383524|
+--------------+------------------+------------------+



In [143]:
write_sql_to_file(comment, query, sql_path)

## f. Which duration bin will you advise a taxi driver to target to maximise his income?

The duration of less than 5 minutes is advised to maximise his/her income on a km per dollar basis.

In [144]:
spark.sparkContext.stop()